In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras import layers, models
# from carbontracker.tracker import CarbonTracker
# tracker = CarbonTracker(epochs=10)



from pyJoules.energy_meter import measure_energy
# @measure_energy



	# ...
	# Instructions to be evaluated.
# print('helloworld')
	# ...



# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

x_train, y_train = preprocess(x_train, y_train)
x_test, y_test = preprocess(x_test, y_test)

# Create the Keras model
@measure_energy
def create_cifar10_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    return model

# Create a TFF version of the Keras model 
def model_fn():
    keras_model = create_cifar10_model() #(HERE U CHANGE THE MODEL)
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=(
            tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

# Simulate federated data by splitting the dataset into multiple clients
def split_data_for_clients(data, client_count):
    client_data = []
    data_len = len(data[0])
    batch_size = data_len // client_count

    for i in range(client_count):
        start = i * batch_size
        end = (i + 1) * batch_size if i != client_count - 1 else data_len
        client_data.append((data[0][start:end], data[1][start:end]))

    return client_data

client_count = 10
client_data = split_data_for_clients((x_train, y_train), client_count)

# Create a federated dataset from the client data
federated_data = [
    tff.simulation.ClientData.from_clients_and_fn(
        client_ids=[str(i)], create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices(client_data[i]).batch(20)
    ).create_tf_dataset_for_client(str(i))
    for i in range(client_count)
]





# tracker.stop()









import nest_asyncio
nest_asyncio.apply()

# Train the federated model
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
)

# Initialize the state
state = trainer.initialize()

# Train the model for multiple rounds
num_rounds = 10
for round_num in range(1, num_rounds + 1):
    print(f"Round {round_num}")
    state, metrics = trainer.next(state, federated_data)
    print(f"Metrics: {metrics}")








    # Convert the test dataset into a list of datasets
client_test_data = split_data_for_clients((x_test, y_test), client_count)

# Create a federated dataset from the client test data
federated_test_data = [
    tf.data.Dataset.from_tensor_slices(client_test_data[i]).batch(len(client_test_data[i][0]))
    for i in range(client_count)
]

# Evaluate the trained model on the federated test dataset
tff_evaluator = tff.learning.build_federated_evaluation(model_fn)
test_metrics = tff_evaluator(state.model, federated_test_data)
print(f"Test Metrics: {test_metrics}")


meter.end()
meter.result

2023-04-29 14:28:19.769717: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.
2023-04-29 14:28:19.912862: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


begin timestamp : 1682771300.1817064; tag : create_cifar10_model; duration : 0.14760804176330566; package_0 : 781859.0; core_0 : 521789.0; uncore_0 : 72632.0
begin timestamp : 1682771300.59314; tag : create_cifar10_model; duration : 0.09064459800720215; package_0 : 579222.0; core_0 : 441833.0; uncore_0 : 33813.0
begin timestamp : 1682771301.0125115; tag : create_cifar10_model; duration : 0.08551526069641113; package_0 : 478637.0; core_0 : 349181.0; uncore_0 : 35583.0
begin timestamp : 1682771301.5180442; tag : create_cifar10_model; duration : 0.43654870986938477; package_0 : 2471063.0; core_0 : 1687068.0; uncore_0 : 196166.0
begin timestamp : 1682771303.6472204; tag : create_cifar10_model; duration : 0.09239006042480469; package_0 : 559997.0; core_0 : 413268.0; uncore_0 : 45227.0
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2023-04-29 14:28:25.740200: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-29 14:28:25.762628: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2803200000 Hz


Round 1
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.13878), ('loss', 2.2802544)])), ('stat', OrderedDict([('num_examples', 50000)]))])
Round 2
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.209), ('loss', 2.1487389)])), ('stat', OrderedDict([('num_examples', 50000)]))])
Round 3
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.256), ('loss', 2.0311894)])), ('stat', OrderedDict([('num_examples', 50000)]))])
Round 4
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.29252), ('loss', 1.945558)])), ('stat',

KeyboardInterrupt: 

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Train the federated model
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
)

# Initialize the state
state = trainer.initialize()

# Train the model for multiple rounds
num_rounds = 10
for round_num in range(1, num_rounds + 1):
    print(f"Round {round_num}")
    state, metrics = trainer.next(state, federated_data)
    print(f"Metrics: {metrics}")


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2023-04-25 16:04:32.020188: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-25 16:04:32.043210: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2803200000 Hz


Round 1


KeyboardInterrupt: 

In [ ]:
# Convert the test dataset into a list of datasets
client_test_data = split_data_for_clients((x_test, y_test), client_count)

# Create a federated dataset from the client test data
federated_test_data = [
    tf.data.Dataset.from_tensor_slices(client_test_data[i]).batch(len(client_test_data[i][0]))
    for i in range(client_count)
]

# Evaluate the trained model on the federated test dataset
tff_evaluator = tff.learning.build_federated_evaluation(model_fn)
test_metrics = tff_evaluator(state.model, federated_test_data)
print(f"Test Metrics: {test_metrics}")

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras import layers, models
import nest_asyncio
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from pyJoules.energy_meter import measure_energy






@measure_energy
def fed_model(num_rounds, clients_per_round ):

    # Load the CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Preprocess the data
    def preprocess(x, y):
        x = tf.cast(x, tf.float32) / 255.0
        y = tf.cast(y, tf.int64)
        return x, y


    x_train, y_train = preprocess(x_train, y_train)
    x_test, y_test = preprocess(x_test, y_test)

    # # Create the Keras model
    # def create_cifar10_model():
    #     model = models.Sequential()
    #     model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    #     model.add(layers.MaxPooling2D((2, 2)))
    #     model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #     model.add(layers.MaxPooling2D((2, 2)))
    #     model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #     model.add(layers.Flatten())
    #     model.add(layers.Dense(64, activation='relu'))
    #     model.add(layers.Dense(10))
    #     return model
    
    def create_vgg16_model():
        # Load the VGG-16 model
        base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))
    
        # Add custom layers on top of the VGG-16 model
        x = Flatten()(base_model.output)
        x = Dense(256, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        predictions = Dense(10)(x)

        # Create the model
        model = Model(inputs=base_model.input, outputs=predictions)
        return model

    # # Create a TFF version of the Keras model 
    # def model_fn():
    #     keras_model = create_cifar10_model() #(HERE U CHANGE THE MODEL)
    #     return tff.learning.from_keras_model(
    #         keras_model,
    #         input_spec=(
    #             tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
    #             tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
    #         ),
    #         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    #     )



    # Create a TFF version of the Keras model 
    def model_fn():
        keras_model = create_vgg16_model()
        return tff.learning.from_keras_model(
            keras_model,
            input_spec=(
                tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
            ),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
        )





    # Simulate federated data by splitting the dataset into multiple clients
    def split_data_for_clients(data, client_count):
        client_data = []
        data_len = len(data[0])
        batch_size = data_len // client_count

        for i in range(client_count):
            start = i * batch_size
            end = (i + 1) * batch_size if i != client_count - 1 else data_len
            client_data.append((data[0][start:end], data[1][start:end]))

        return client_data

    client_count = 10
    client_data = split_data_for_clients((x_train, y_train), client_count)

    # Create a federated dataset from the client data
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)], create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices(client_data[i]).batch(20)
        ).create_tf_dataset_for_client(str(i))
        for i in range(client_count)
    ]


    nest_asyncio.apply()

    # Train the federated model
    trainer = tff.learning.build_federated_averaging_process(
        model_fn,
        client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
        # client_optimizer_fn=client_optimizer,
        # server_optimizer_fn=server_optimizer,
        client_weighting=tff.learning.ClientWeighting.NUM_EXAMPLES
    )

    # Initialize the state
    state = trainer.initialize()

    # Train the model for multiple rounds
    # num_rounds = 10
    for round_num in range(1, num_rounds + 1):
        print(f"Round {round_num}")
        state, metrics = trainer.next(state, federated_data) #?????????????????????????????????????????????????????????????????????????????????????????????????????
        print(f"Metrics: {metrics}") #????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
        # Select a random subset of clients to participate in this round
        client_ids = np.random.choice(client_count, clients_per_round, replace=False)
        client_data_for_round = [federated_data[i] for i in client_ids]

        # Train the model on the selected clients' data
        state, metrics = trainer.next(state, client_data_for_round)
        print(f"Metrics: {metrics}")



    # Convert the test dataset into a list of datasets
    client_test_data = split_data_for_clients((x_test, y_test), client_count)

    # Create a federated dataset from the client test data
    federated_test_data = [
        tf.data.Dataset.from_tensor_slices(client_test_data[i]).batch(len(client_test_data[i][0]))
        for i in range(client_count)
    ]

    # Evaluate the trained model on the federated test dataset
    tff_evaluator = tff.learning.build_federated_evaluation(model_fn)
    test_metrics = tff_evaluator(state.model, federated_test_data)
    print(f"Test Metrics: {test_metrics}")
    return test_metrics

vgg = fed_model(10, 5)
vgg

2023-05-01 08:27:15.117356: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-01 08:27:15.118106: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-01 08:27:15.118117: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-01 08:27:15.118132: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gaia-XPS-13-9310): /proc/driver/nvidia/version does not exist
2023-05-01 08:27:15.118683: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2023-05-01 08:27:19.381655: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-05-01 08:27:19.413478: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2803200000 Hz


Round 1
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.15592), ('loss', 2.2329133)])), ('stat', OrderedDict([('num_examples', 50000)]))])
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.23264), ('loss', 2.0789487)])), ('stat', OrderedDict([('num_examples', 25000)]))])
Round 2
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.26964), ('loss', 2.0018804)])), ('stat', OrderedDict([('num_examples', 50000)]))])
Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.30556), ('loss', 1.923649)])), ('stat', OrderedDict

OrderedDict([('sparse_categorical_accuracy', 0.5761), ('loss', 1.1962435)])

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import nest_asyncio
import torch
import torchvision.transforms as transforms
from timm.models import create_model
from pyJoules.energy_meter import measure_energy






@measure_energy
def fed_model(num_rounds, clients_per_round ):

    # Load the CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Preprocess the data
    def preprocess(x, y):
        x = tf.cast(x, tf.float32) / 255.0
        x = tf.image.resize(x, (128, 128))
        y = tf.cast(y, tf.int64)
        return x, y


    x_train, y_train = preprocess(x_train, y_train)
    x_test, y_test = preprocess(x_test, y_test)

    # # Create the Keras model
    # def create_cifar10_model():
    #     model = models.Sequential()
    #     model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    #     model.add(layers.MaxPooling2D((2, 2)))
    #     model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #     model.add(layers.MaxPooling2D((2, 2)))
    #     model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #     model.add(layers.Flatten())
    #     model.add(layers.Dense(64, activation='relu'))
    #     model.add(layers.Dense(10))
    #     return model
    
    # def create_vgg16_model():
    #     # Load the VGG-16 model
    #     base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))
    
    #     # Add custom layers on top of the VGG-16 model
    #     x = Flatten()(base_model.output)
    #     x = Dense(256, activation='relu')(x)
    #     x = Dense(128, activation='relu')(x)
    #     predictions = Dense(10)(x)

    #     # Create the model
    #     model = Model(inputs=base_model.input, outputs=predictions)
    #     return model

    # # Create a TFF version of the Keras model 
    # def model_fn():
    #     keras_model = create_cifar10_model() #(HERE U CHANGE THE MODEL)
    #     return tff.learning.from_keras_model(
    #         keras_model,
    #         input_spec=(
    #             tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
    #             tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
    #         ),
    #         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    #     )


    # Convert the dataset to PyTorch tensors
    x_train_pt = torch.from_numpy(np.transpose(x_train, (0, 3, 1, 2)))
    y_train_pt = torch.from_numpy(y_train.squeeze())
    x_test_pt = torch.from_numpy(np.transpose(x_test, (0, 3, 1, 2)))
    y_test_pt = torch.from_numpy(y_test.squeeze())

    # Create the ViT model using timm
    def create_vit_model():
        model = create_model('vit_tiny_patch16_224', pretrained=True)
        model.head = torch.nn.Linear(model.head.in_features, 10)
        return model

    # Convert the ViT model to TensorFlow format
    def convert_to_tf_model(vit_model):
        input_shape = (None, 128, 128, 3)
        img = tf.keras.Input(shape=input_shape[1:], dtype=tf.float32)
        x = tf.keras.applications.vgg19.preprocess_input(img)
        x = tf.image.resize(x, (224, 224))
        outputs = vit_model(x)
        outputs = tf.squeeze(outputs[:, 0, :])
        return tf.keras.Model(inputs=img, outputs=outputs)



    # Create a TFF version of the Keras model 
    def model_fn():
        vit_model = create_vit_model()
        tf_model = convert_to_tf_model(vit_model)
        return tff.learning.from_keras_model(
            tf_model,
            input_spec=(
                tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )


def split_data_for_clients(data, client_count, batch_size):
    client_data = []
    data_len = len(data[0])
    samples_per_client = data_len // client_count

    for i in range(client_count):
        start = i * samples_per_client
        end = (i + 1) * samples_per_client
        x = data[0][start:end]
        y = data[1][start:end]
        client_data.append(tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size))

    return client_data
    client_count = 10

    client_data = split_data_for_clients((x_train, y_train), client_count, batch_size=10)



    # # Simulate federated data by splitting the dataset into multiple clients
    # def split_data_for_clients(data, client_count):
    #     client_data = []
    #     data_len = len(data[0])
    #     batch_size = data_len // client_count

    #     for i in range(client_count):
    #         start = i * batch_size
    #         end = (i + 1) * batch_size if i != client_count - 1 else data_len
    #         client_data.append((data[0][start:end], data[1][start:end]))

    #     return client_data

    # client_count = 10
    # client_data = split_data_for_clients((x_train_pt, y_train_pt), client_count)

    # Create a federated dataset from the client data
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)], create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices(client_data[i]).batch(20)
        ).create_tf_dataset_for_client(str(i))
        for i in range(client_count)
    ]


    nest_asyncio.apply()

    # Train the federated model
    trainer = tff.learning.build_federated_averaging_process(
        model_fn,
        client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
        # client_optimizer_fn=client_optimizer,
        # server_optimizer_fn=server_optimizer,
        client_weighting=tff.learning.ClientWeighting.NUM_EXAMPLES
    )

    # Initialize the state
    state = trainer.initialize()

    # Train the model for multiple rounds
    # num_rounds = 10
    for round_num in range(1, num_rounds + 1):
        print(f"Round {round_num}")
        state, metrics = trainer.next(state, federated_data) #?????????????????????????????????????????????????????????????????????????????????????????????????????
        print(f"Metrics: {metrics}") #????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
        # Select a random subset of clients to participate in this round
        client_ids = np.random.choice(client_count, clients_per_round, replace=False)
        client_data_for_round = [federated_data[i] for i in client_ids]

        # Train the model on the selected clients' data
        state, metrics = trainer.next(state, client_data_for_round)
        print(f"Metrics: {metrics}")



    # Convert the test dataset into a list of datasets
    client_test_data = split_data_for_clients((x_test_pt, y_test_pt), client_count)

    # Create a federated dataset from the client test data
    federated_test_data = [
        tf.data.Dataset.from_tensor_slices(client_test_data[i]).batch(len(client_test_data[i][0]))
        for i in range(client_count)
    ]

    # Evaluate the trained model on the federated test dataset
    tff_evaluator = tff.learning.build_federated_evaluation(model_fn)
    test_metrics = tff_evaluator(state.model, federated_test_data)
    print(f"Test Metrics: {test_metrics}")
    return test_metrics

vit = fed_model(10, 5)
vit

/home/gaia/anaconda3/envs/jj1.3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-04 00:30:34.926330: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 9830400000 exceeds 10% of free system memory.


: 

: 

In [1]:
# import tensorflow as tf
# num_epochs = 10


# tf.keras.mixed_precision.set_global_policy('mixed_float16')
# optimizer = tf.keras.optimizers.Adam()
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
# val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# @tf.function
# def train_step(model, inputs, labels):
#     with tf.GradientTape() as tape:
#         logits = model(inputs, training=True)
#         loss = loss_fn(labels, logits)
#         scaled_loss = optimizer.get_scaled_loss(loss)
#     scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
#     gradients = optimizer.get_unscaled_gradients(scaled_gradients)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     train_acc_metric.update_state(labels, logits)

# @tf.function
# def val_step(model, inputs, labels):
#     logits = model(inputs, training=False)
#     val_acc_metric.update_state(labels, logits)

# for epoch in range(num_epochs):
#     train_acc_metric.reset_states()
#     for batch in train_data:
#         train_step(model, *batch)
#     train_acc = train_acc_metric.result()
#     val_acc_metric.reset_states()
#     for batch in val_data:
#         val_step(model, *batch)
#     val_acc = val_acc_metric.result()
#     print(f"Epoch {epoch + 1}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")
